In [1]:
##Commencement : importer les packages

import requests 
import pandas as pd
import json   
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import re


In [2]:
website = "https://www.bymycar.fr/v/voiture-occasion?itemsperpage=24&sortingorder=ascending&filter=marque-du-vehicule%3Amercedes"

In [3]:
response = requests.get(website)

In [4]:
soup = BeautifulSoup(response.content,"html.parser")

In [5]:
results = soup.find_all("a",{"class" : "product-item" })

In [6]:
len(results)

24

In [7]:
############## création boucle pour récupérer sur le véhicule :


# Marque 

# Modele

# Info vehicule 

# Prix

# Nombre de kilométrage 

# Année 

# puissance_chevaux


### Création d'une boucle pour récupérer les informations en ciblant les balises et les classes

In [8]:
modele = []
detail = []
prix = []
type_v = []
boite_v = []

for i in range(1,20):
    
    # website dans une variable
    website = "https://www.bymycar.fr/v/voiture-occasion?filter=marque-du-vehicule%3Amercedes&sortingorder=ascending&itemsperpage=24&display=grid&pagenumber="+str(i)

    # resquest au website
    response = requests.get(website)
    
    # soup object
    soup = BeautifulSoup(response.content,"html.parser")
    
    #result
    results = soup.find_all("a",{"class" : "product-item"})
    

    for result in results :

      try:

        modele.append(result.find("h2",{"class":"product__title"}).get_text())

      except:
        modele.append('n/a')

      try:

        detail.append(result.find("p",{"class" : "product__type" }).get_text())

      except:
        detail.append('n/a')  

      try:

        prix.append(result.find("div",{"class":"product__price_current"}).get_text())

      except:
        prix.append('n/a')

      try:

        type_v.append(result.find("ul",{"class" : "product__characteristics" }).get_text())

      except:
        type_v.append('n/a')      

      try:

        boite_v.append(result.find("ul",{"class" : "product__features_list" }).get_text())

      except:
        boite_v.append('n/a')    


In [9]:
car_dealer = pd.DataFrame({"Modele":modele , "Info vehicule":detail ,"Prix" :prix,
                           "type_vehicule":type_v,"puissance":boite_v})

### clean les colonnes du dataframe
-------------- Nettoyage de la base de données :


In [10]:
car_dealer["type_vehicule"] = car_dealer["type_vehicule"].str.lstrip("\n")

In [11]:
car_dealer["Nombre de kilometre"] = car_dealer["type_vehicule"].str.split("km", expand = True)[0]

In [12]:
def extract_after_word(string, word):
    if word in string:
        return string[string.index(word)+len(word):string.index(word)+len(word)+16]
    else:
        return None

In [13]:
car_dealer['Annee_sortie'] = car_dealer['type_vehicule'].apply(lambda x: extract_after_word(x,"km"))

In [14]:
def keep_digits(string):
    return ''.join(filter(str.isdigit, string))

In [15]:
car_dealer['Annee_sortie'] = car_dealer['Annee_sortie'].apply(keep_digits)

In [16]:
del car_dealer["type_vehicule"]

In [17]:
def extract_before_word(string, word):
    if word in string:
        return string[string.index(word)-4:string.index(word)]
    else:
        return None

In [18]:
car_dealer['puissance_chevaux'] = car_dealer['puissance'].apply(lambda x: extract_before_word(x, "cv"))

In [19]:
del car_dealer["puissance"]

In [20]:
car_dealer['Marque'] = "Mercedes"

In [21]:
car_dealer = car_dealer.reindex(columns=['Marque', 'Modele', 'Info vehicule','Prix', 'Nombre de kilometre', 'Annee_sortie',
                                         'puissance_chevaux'])


In [22]:
car_dealer['Prix'] = car_dealer['Prix'].apply(keep_digits)

In [23]:
car_dealer

,Marque,Modele,Info vehicule,Prix,Nombre de kilometre,Annee_sortie,puissance_chevaux
0,Mercedes,Classe A 200 d 8G-DCT,DIESEL,46550,0,2022,150
1,Mercedes,Classe E 63 S AMG Speedshift TCT AMG 4-Matic+,Essence,95569,28084,2020,612
2,Mercedes,Classe A 200 7G-DCT,Essence,32480,41096,2019,163
3,Mercedes,Classe CLS Shooting Brake 220 d,DIESEL,23282,100822,2017,170
4,Mercedes,GLC 300 de 9G-Tronic 4Matic,Hybride,72900,10000,2022,194
...,...,...,...,...,...,...,...
244,Mercedes,Classe A 180 d 8G-DCT,DIESEL,35900,10000,2022,116
245,Mercedes,Classe V Long 250 d 9G-TRONIC,DIESEL,79900,100,2022,190
246,Mercedes,GLE Coupé 43 AMG 9G-Tronic 4MATIC,Essence,58647,46399,2017,367
247,Mercedes,G 63 AMG BVA9,Essence,274990,14990,2022,None


In [25]:
#exportation sous excel
#car_dealer.to_excel("Bymycar_excel.xlsx", index= False)